In [1]:
from datasets import load_dataset
import json

/home/hatakeyama/miniconda3/envs/llmeval/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#name="team-hatakeyama-phase2/Synthetic-JP-EN-Coding-Dataset-1.25M"
#ds=load_dataset(name,split="train")

In [11]:

dataset_name_list=[
    ("team-hatakeyama-phase2/synth-persona-jp-math-nemotron-4","train",10**4),
    ("team-hatakeyama-phase2/ChatBotLikeArena","20240728",10**4),
    ("team-hatakeyama-phase2/Synthetic-JP-EN-Coding-Dataset-1.25M","train",2*10**4),
]

In [12]:
i=0
question_template="以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。\n\n### 指示:\n"
answer_template="\n### 応答:\n"
regen_record_list=[]
for ds_name,split,n_records in dataset_name_list:
    ds=load_dataset(ds_name,split=split)
    ds=ds.shuffle()

    cnt=0
    for record in ds:
        cnt+=1
        if cnt>n_records:
            break

        if "messages" in record:
            q=record["messages"][0]["content"]
            a=record["messages"][1]["content"]
        elif "prompt" in record:
            q=record["prompt"]
            a=record["chosen"]
        else:
            raise Exception("unknown record format: "+str(record))

        d={
        "instruction":f"{question_template}{q}{answer_template}",
        "output":a, 
        }
        regen_record_list.append(d)


In [17]:
import random
random.shuffle(regen_record_list)

In [18]:
import datasets

regen_ds=datasets.Dataset.from_list(regen_record_list)

In [19]:
regen_ds.save_to_disk("out/regen_master_ds")

Saving the dataset (1/1 shards): 100%|██████████| 31321/31321 [00:00<00:00, 234746.12 examples/s]


In [20]:
regen_ds[0]

{'instruction': '以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。\n\n### 指示:\n小学校の教頭先生は、毎月、学校の全児童に飴を配ります。今月は、全校児童の数が先月より10人増え、計350人になりました。教頭先生は、一人当たり飴を2個ずつ配ります。しかし、飴の入った箱を開けてみると、教頭先生の勘違いで、実際には必要数より20個少ない数の飴しか入っていませんでした。\n\n(1) 最初に、教頭先生は全部で何個の��\n\n...\n\nMath problem:\n\n小学校の教頭先生は、毎月、学校の全児童に飴を配ります。今月は、全校児童の数が先月より10人増え、計350人になりました。教頭先生は、一人当たり飴を2個ずつ配ります。しかし、飴の入った箱を開けてみると、教頭先生の勘違いで、実際には必要数より20個少ない数の飴しか入っていませんでした。\n\n(1) 最初に、教頭先生は全部で何個の飴を準備する予定でしたか？\n\n(2) 足りない飴の数を、教頭先生は学校にある飴を補充する箱から補充することにしました。その飴を補充する箱には、白い飴と赤い飴が混ざって入っており、白い飴と赤い飴の数の比は2:3でした。教頭先生は児童に配布する飴の色のバランスを考えて、不足している飴20個の半分を白い飴、もう半分を赤い飴で補充することにしました。教頭先生は飴を補充する箱から白い飴と赤い飴をそれぞれ何個補充しましたか？\n### 応答:\n',
 'output': '(1) 教頭先生は、一人当たり飴を2個ずつ配る予定なので、全校児童の数である350人と飴の個数をかけると、全部で700個の飴を準備する予定でした。\n\n(2) 足りない飴の数は20個です。教頭先生は不足している飴の半分を白い飴、もう半分を赤い飴で補充することにしたので、それぞれ10個補充することになります。飴を補充する箱には白い飴と赤い飴が混ざって入っており、白い飴と赤い飴の数の比は2:3なので、白い飴と赤い飴の総数は5つの倍数になっています。したがって、白い飴を10個補充するには、赤い飴を15個補充する必要があります。しかし、赤い飴は10個しか補充しない予定なので、白い飴を予定より少なく補充するか、赤い飴を予定より多く補充する必要があります

In [21]:
len(regen_ds)

31321